# Simplified TextRank
- using class

In [35]:
import numpy as np
import pandas as pd

In [22]:
def make_pairs(tokens, window=2):
    
    nodes = list(set(tokens))
    vocab = nodes
    vocab2idx = {k:i for (i,k) in enumerate(nodes)}
    idx2vocab = {i:k for (i,k) in enumerate(nodes)}
    
    idx = list(idx2vocab.keys())
    #print(idx)
    
    res_dict = {k:[] for k in vocab}

    for i, token in enumerate(tokens):
        #print(token)
        leng = len(tokens)
        
        min_idx = max(0, i-window+1)
        max_idx = min(leng, i+window)
        
        for t in tokens[min_idx:max_idx]:  
            if t != token:
                res_dict[token].append(t)
    
    #print('with duplicates', res_dict)
    res_dict = {k:list(set(v)) for k, v in res_dict.items()}
    return res_dict

In [24]:
class trNode:
    def __init__(self, name, damping_factor=0.85):
        self.name = name
        self.d = damping_factor
        self.fnodes = []
        
        # initial score
        self.score = 1
        
    def update_score(self):
        # update node score
        input_weights = [fnode.score/len(fnode.fnodes) for fnode in self.fnodes]
        self.score = (1-self.d) + self.d * sum(input_weights)
        
    def update_fnodes(self, nodes):
        for node in nodes:
            self.fnodes.append(node)

In [25]:
class textRank:
    def __init__(self, nodes):
        self.nodes = nodes
        
    def update_nodes(self, times=None, threshold = 0.0001):
        # limited times
        if times:
            for i in range(times):
                [n.update_score() for n in self.nodes]
                return [(n.name, n.score) for n in self.nodes]
        # until convergence (< threshold)
        else:
            diff_scores = np.ones(len(self.nodes))
            while not (diff_scores < threshold).all():
                current_scores = np.array([n.score for n in self.nodes])
                [n.update_score() for n in self.nodes]
                new_scores = np.array([[n.score for n in self.nodes]])
                diff_scores = np.abs(new_scores - current_scores)
            return [(n.name, n.score) for n in self.nodes]        

In [31]:
# step1. convert a list of tokens into a dicionary of window information
tokens = ['딸기', '바나나', '사과', '딸기', '파인애플']
window_dict = make_pairs(tokens, window=2)

# step2. update fnodes to each textrank node
name2node = {name:trNode(name) for name in list(window_dict.keys())}
#print(name2node)
for node_name, node_fnodes in window_dict.items():
    fnodes_ = [name2node[fnode] for fnode in node_fnodes]
    name2node[node_name].update_fnodes(fnodes_)

# step3. calculate node weight
tr = textRank(list(name2node.values()))
tr.update_nodes()

[('딸기', 1.4671812436569796),
 ('파인애플', 0.5657013523694776),
 ('사과', 0.9838537217994482),
 ('바나나', 0.983839184134243)]

### Function wrapping

In [45]:
def compute_textrank(tokens, window=2):
    # step1. convert a list of tokens into a dicionary of window information
    window_dict = make_pairs(tokens, window)

    # step2. update fnodes to each textrank node
    name2node = {name:trNode(name) for name in list(window_dict.keys())}
    #print(name2node)
    for node_name, node_fnodes in window_dict.items():
        fnodes_ = [name2node[fnode] for fnode in node_fnodes]
        name2node[node_name].update_fnodes(fnodes_)

    # step3. calculate node weight
    tr = textRank(list(name2node.values()))
    scores = tr.update_nodes()
    df_scores_sorted = pd.DataFrame(scores,
             columns = ['token', 'weight']).sort_values(by='weight', ascending=False)
    
    return df_scores_sorted

In [46]:
test_article = '''
과기정통부 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도 개방성 강화 이데일리 이재운 기자 국가 차원의 빅데이터 활용 시대가 열린다 새로운 산업 창출과 기존 산업의 변화에 이르는 혁신성장 을 위한 센터가 문을 연다 10개 분야에 걸쳐 데이터 경제 의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다 22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 빅데이터 플랫폼 및 센터 출범식 행사를 개최했다 유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다 10개 분야 100개 센터 3년간 1516억원 투입이미지 픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다 문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호 보안 에 대한 중요성을 강조하기도 했다 이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산 구축하고 플랫폼은 이를 수집 분석 유통하는 역할을 담당한다 과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며 올해 우선 640억원 규모의 사업을 추진하고 있다 대상 분야는 금융 BC카드 환경 한국수자원공사 문화 한국문화정보원 교통 한국교통연구원 헬스케어 국립암센터 유통 소비 매일방송 통신 KT 중소기업 더존비즈온 지역경제 경기도청 산림 한국임업진흥원 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원 운영할 계획이다 이를 통해 데이터 생태계를 혁신하고 기업의 경쟁력을 제고하는 역할을 수행한다 주요 활용 전략 사례를 보면 빅데이터 활용을 통해 신 시장 을 창출하는 방안을 담고 있다 금융 플랫폼의 경우 소상공인 신용평가 고도화 등을 통해 금융 취약 계층 대상 중금리 대출이자를 2%p 절감해 연간 1조원의 신규대출을 창출할 전망이다 유통 소비와 중소기업 플랫폼은 소상공인이나 중소기업의 폐업률 감소를 문화 플랫폼은 문화 예술 관람률과 생활체육 참여율을 높이는 방안을 모색한다 의료비 절감 헬스케어 과 기업의 매출 향상을 통한 산업 육성 통신 산림 등도 눈길을 끈다 과기정통부 제공 2021년까지 5100여종 데이터 구축 AI 알고리즘 제공도센터는 우선 분야별 데이터 부족 문제를 해소하기 위해 올해 말까지 시장 수요가 높은 1400여종 신규 데이터를 생산 구축하고 사업이 완료되는 2021년까지 총 5100여종 양질의 풍부한 데이터를 생산 구축해 시장에 공급할 계획이다 특히 공공과 민간 사이 데이터 파일형식 등이 달라 호환이 제대로 이뤄지지 못한 문제를 해소하기 위해 개방형 표준을 적용하고 품질관리기준도 마련해 운영한다 기업들이 실제 활용 가능한 최신 데이터를 확보하는데도 수개월이 소요된다는 문제점을 개선하기 위한 방안도 추진한다 센터와 플랫폼 간 연계체계에는 민간 클라우드를 기반으로 활용하고 센터에 축적된 데이터도 계속 외부와 개방 공유하며 최신 연속성을 확보한다는 계획이다 100개 센터에서 수집된 데이터를 융합 분석한 뒤 맞춤형 데이터 제작 등 양질의 데이터로 재생산하고 기업들이 필요로 하는 데이터를 원하는 형태로 즉시 활용할 수 있도록 제공할 계획이다 다양한 분석 도구는 물론 인공지능 AI 학습 알고리즘도 제공해 이용자가 보다 사용하기 편리한 환경을 제공한다 이밖에 필요한 데이터를 쉽게 등록하고 검색할 수 있도록 기준을 마련하고 데이터 보유와 관리에 대한 체계 거버넌스 를 논의하는 데이터 얼라이언스 를 구성해 보다 안전하게 이용하는 방안도 마련했다 유영민 과기정통부 장관은 오늘 출범식은 대한민국이 데이터 강국으로 가기 위한 초석을 놓은 자리 라며 세계 주요국들보다 데이터 경제로 나아가는 발걸음이 다소 늦었지만 빅데이터 플랫폼과 센터를 지렛대로 우리나라의 낙후된 데이터 생태계를 혁신하고 기업의 경쟁력을 한 단계 제고할 수 있도록 정책적 역량을 집중하겠다 고 밝혔다 이재운
'''
test_article_tokens = test_article.split()
article_textrank = compute_textrank(test_article_tokens)
article_textrank[:10]

,token,weight
179,데이터,9.027626
132,데이터를,4.007262
232,위한,3.312292
251,계획이다,3.258675
331,빅데이터,3.171702
43,통해,3.162752
137,활용,2.075149
296,금융,2.067445
168,과기정통부,2.057109
127,등,2.030982


In [48]:
article_textrank = compute_textrank(test_article_tokens,
                                   window=5)
article_textrank[:10]

,token,weight
179,데이터,9.181882
132,데이터를,4.340775
43,통해,3.751861
251,계획이다,3.414695
232,위한,3.342863
331,빅데이터,3.232621
168,과기정통부,2.292451
127,등,2.109018
137,활용,2.076501
11,플랫폼,2.060321


# TextRank using NetworkX

In [51]:
import networkx as nx

In [53]:
window_dict = make_pairs(tokens, 2)
window_dict

{'딸기': ['바나나', '파인애플', '사과'],
 '파인애플': ['딸기'],
 '사과': ['바나나', '딸기'],
 '바나나': ['딸기', '사과']}

In [57]:
node_pairs = [(k,vv) for k,v in window_dict.items() for vv in v]
node_pairs

[('딸기', '바나나'),
 ('딸기', '파인애플'),
 ('딸기', '사과'),
 ('파인애플', '딸기'),
 ('사과', '바나나'),
 ('사과', '딸기'),
 ('바나나', '딸기'),
 ('바나나', '사과')]

In [61]:
graph = nx.diamond_graph()
graph.clear()

vocab = list(window_dict.keys())
graph.add_nodes_from(vocab)
graph.add_edges_from(node_pairs)

In [62]:
scores = nx.pagerank(graph)

In [63]:
scores

{'딸기': 0.3667352990529792,
 '파인애플': 0.14140875554444032,
 '사과': 0.2459279727012903,
 '바나나': 0.24592797270129033}

In [78]:
window_dict = make_pairs(test_article.split(), 2)
node_pairs = [(k,vv) for k,v in window_dict.items() for vv in v]

graph = nx.diamond_graph()
graph.clear()

vocab = list(window_dict.keys())
graph.add_nodes_from(vocab)
graph.add_edges_from(node_pairs)
scores = nx.pagerank(graph)

pd.DataFrame(scores, index=[0]).T.sort_values(by=0, ascending=False)[:10]

,0
데이터,0.024196
데이터를,0.010742
위한,0.008880
계획이다,0.008735
빅데이터,0.008502
통해,0.008479
활용,0.005563
금융,0.005542
과기정통부,0.005515
등,0.005444


# Simplified TextRank 2
- applying `window_size`

In [5]:
tokens = ['딸기', '바나나', '사과', '딸기', '파인애플']

nodes = list(set(tokens))
vocab = nodes
vocab2idx = {k:i for (i,k) in enumerate(nodes)}
idx2vocab = {i:k for (i,k) in enumerate(nodes)}

In [6]:
vocab2idx

{'딸기': 0, '파인애플': 1, '사과': 2, '바나나': 3}

In [7]:
idx2vocab

{0: '딸기', 1: '파인애플', 2: '사과', 3: '바나나'}

In [211]:
tokens

['딸기', '바나나', '사과', '딸기', '파인애플']

In [17]:
def make_pairs(tokens, window=2):
    
    nodes = list(set(tokens))
    vocab = nodes
    vocab2idx = {k:i for (i,k) in enumerate(nodes)}
    idx2vocab = {i:k for (i,k) in enumerate(nodes)}
    
    idx = list(idx2vocab.keys())
    #print(idx)
    
    res_dict = {k:[] for k in vocab}

    for i, token in enumerate(tokens):
        #print(token)
        leng = len(tokens)
        
        min_idx = max(0, i-window+1)
        max_idx = min(leng, i+window)
        
        for t in tokens[min_idx:max_idx]:  
            if t != token:
                res_dict[token].append(t)
    
    print('with duplicates', res_dict)
    res_dict = {k:list(set(v)) for k, v in res_dict.items()}
    return res_dict

In [13]:
tokens

['딸기', '바나나', '사과', '딸기', '파인애플']

In [19]:
token_pairs = make_pairs(tokens)
token_pairs

with duplicates {'딸기': ['바나나', '사과', '파인애플'], '파인애플': ['딸기'], '사과': ['바나나', '딸기'], '바나나': ['딸기', '사과']}


{'딸기': ['바나나', '파인애플', '사과'],
 '파인애플': ['딸기'],
 '사과': ['바나나', '딸기'],
 '바나나': ['딸기', '사과']}

In [18]:
tokens2 = ['딸기', '바나나', '사과', '딸기', '파인애플', '바나나', '딸기', '사과', '바나나']
make_pairs(tokens2)

with duplicates {'딸기': ['바나나', '사과', '파인애플', '바나나', '사과'], '파인애플': ['딸기', '바나나'], '사과': ['바나나', '딸기', '딸기', '바나나'], '바나나': ['딸기', '사과', '파인애플', '딸기', '사과']}


{'딸기': ['바나나', '파인애플', '사과'],
 '파인애플': ['딸기', '바나나'],
 '사과': ['바나나', '딸기'],
 '바나나': ['딸기', '파인애플', '사과']}

In [249]:
strawberry_fnodes = token_pairs['딸기']
banana_fnodes = token_pairs['바나나']
apple_fnodes = token_pairs['사과']
pineapple_fnodes = token_pairs['파인애플']

In [251]:
strawberry_score = 1
banana_score = 1
apple_score = 1
pineapple_score = 1

In [305]:
def update_score(t_pairs):
    
    token_names = list(t_pairs.keys())
    token_lengths = [len(v) for v in list(t_pairs.values())]
    
    token_scores = [1] * len(token_names)
    token_powers = [token_scores[i]/len(t_pairs[token]) for (i, token)\
                   in enumerate(token_names)]
    name_power_dict = {n:p for n,p in zip(token_names, token_powers)}
    
    diff_scores = np.array([1, 1, 1, 1])
    
    # update score
    while not np.abs(diff_scores < 0.0001).all():
        # update information
        token_powers = [token_scores[i]/len(t_pairs[token]) for (i, token)\
                   in enumerate(token_names)]
        name_power_dict = {n:p for n,p in zip(token_names, token_powers)}
        prev_scores = token_scores.copy()
        
        for i, (k, v) in enumerate(t_pairs.items()):
            #print(k, v)
            tot = 0
            for node in v:
                tot += (name_power_dict[node])
            token_scores[i] = tot
        
        diff_scores = np.array(token_scores) - np.array(prev_scores)

#        diff_scores = np.array(prev_scores) - np.array(token_scores)
    
    return token_scores

In [306]:
update_score(token_pairs)

[1.000024679409045, 1.000024679409045, 1.4999090282268144, 0.5000416129550959]

# Simplified TextRank 3
- tutor version
    - window sliding

In [ ]:
import numpy as np
import math

In [318]:
vocab_len = len(vocab)
weighted_edge = np.zeros((vocab_len, vocab_len),
                        dtype=np.float32)
score = np.ones((vocab_len), dtype=np.float32)
window_size = 3
covered_co_occurrences = []

In [319]:
for window_start in range(0, len(tokens) - window_size + 1):
    window = tokens[window_start : window_start + window_size]
    print(window)
    for i in range(window_size):
        for j in range(i+1, window_size):
            if (window[i] in vocab2idx.keys()) and (window[j] in vocab2idx.keys()):
                index_of_i = window_start + i
                index_of_j = window_start + j
                
                if [index_of_i, index_of_j] not in covered_co_occurrences:
                    weighted_edge[vocab2idx[window[i]]][vocab2idx[window[j]]] = 1
                    weighted_edge[vocab2idx[window[i]]][vocab2idx[window[j]]] = 1
                    covered_co_occurrences.append([index_of_i, index_of_j])        
    break

['딸기', '바나나', '사과']


In [320]:
weighted_edge

array([[0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [1., 1., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)